In [2]:
import requests
import flair
import pandas as pd
import re


In [3]:
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAL%2F7TwEAAAAAv4pYST1v33FQwb9GVNrG1gHI20w%3DeEqAOC4D7nmI8X2r9YPjLEY4czJ7qw253dLJjbU0iAE0eSeZEn'

In [4]:
params = {
    'q': 'ibovespa',
    'tweet_mode': 'extended',
    'lang': 'pt',
    'count': '100'
}

In [5]:
response = requests.get(
    'https://api.twitter.com/1.1/search/tweets.json',
    params=params,
    headers={'authorization': 'Bearer '+BEARER_TOKEN})

In [6]:
def get_data(tweet):
    data = {
        'id': tweet['id_str'],
        'created_at': tweet['created_at'],
        'text': tweet['full_text']
    }
    return data

In [7]:
df = pd.DataFrame()
for tweet in response.json()['statuses']:
    row = get_data(tweet)
    df = df.append(row, ignore_index=True)

In [8]:
df.head()

,id,created_at,text
0,1441444124427632648,Fri Sep 24 16:47:14 +0000 2021,RT @brasigois: Grupo de milicianos liderados p...
1,1441444083998748677,Fri Sep 24 16:47:05 +0000 2021,RT @FarolLiber: Independente de qual seja a su...
2,1441443423546970117,Fri Sep 24 16:44:27 +0000 2021,"Em dia que o Ibovespa cai, destacam-se as açõe..."
3,1441441821515821057,Fri Sep 24 16:38:05 +0000 2021,"📉Ibovespa cai 0,96%, aos 112.949 pontos\n\n📈Dó..."
4,1441437475151437825,Fri Sep 24 16:20:49 +0000 2021,o índice Ibovespa acusa o contrário (não que e...


In [9]:
def clean(df):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    tesla = re.compile(r"(?i)@Tesla(?=\b)")
    user = re.compile(r"(?i)@[a-z0-9_]+")

    tweet = whitespace.sub('', tweet)
    tweet = web_address.sub('', tweet)
    tweet = tesla.sub('Tesla', tweet)
    tweet = user.sub('', tweet)

In [10]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-09-24 14:00:47,228 loading file C:\Users\pcosm\.flair\models\sentiment-en-mix-distillbert_4.pt


In [11]:
for tweet in df.text:
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)

In [12]:
probability = sentence.labels[0].score  # numerical value 0-1
sentiment = sentence.labels[0].value  # 'POSITIVE' or 'NEGATIVE'

In [13]:
# we will append probability and sentiment preds later
probs = []
sentiments = []

# use regex expressions (in clean function) to clean tweets

for tweet in df['text'].to_list():
    # make prediction
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)
    # extract sentiment prediction
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

# add probability and sentiment predictions to tweets dataframe
df['probability'] = probs
df['sentiment'] = sentiments

In [14]:
df

,id,created_at,text,probability,sentiment
0,1441444124427632648,Fri Sep 24 16:47:14 +0000 2021,RT @brasigois: Grupo de milicianos liderados p...,0.997018,NEGATIVE
1,1441444083998748677,Fri Sep 24 16:47:05 +0000 2021,RT @FarolLiber: Independente de qual seja a su...,0.928126,POSITIVE
2,1441443423546970117,Fri Sep 24 16:44:27 +0000 2021,"Em dia que o Ibovespa cai, destacam-se as açõe...",0.983904,POSITIVE
3,1441441821515821057,Fri Sep 24 16:38:05 +0000 2021,"📉Ibovespa cai 0,96%, aos 112.949 pontos\n\n📈Dó...",0.615970,NEGATIVE
4,1441437475151437825,Fri Sep 24 16:20:49 +0000 2021,o índice Ibovespa acusa o contrário (não que e...,0.629492,POSITIVE
...,...,...,...,...,...
95,1441388721245331461,Fri Sep 24 13:07:05 +0000 2021,RT @FarolLiber: Independente de qual seja a su...,0.928126,POSITIVE
96,1441388498691534849,Fri Sep 24 13:06:12 +0000 2021,RT @brasigois: Grupo de milicianos liderados p...,0.997018,NEGATIVE
97,1441387460882165768,Fri Sep 24 13:02:05 +0000 2021,Esse pessoal do MTST e tão burro... que invadi...,0.993776,POSITIVE
98,1441386891278831619,Fri Sep 24 12:59:49 +0000 2021,@NArgumentos @ajulysantos Ok entendo seu ponto...,0.983842,POSITIVE


In [43]:
df.iloc[95].text

'RT @stem_feed: @Tesla @elonmusk Meanwhile solar roof in Russia https://t.co/3yGqeVPXTA'

In [1]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import requests
import tweepy
from textblob import TextBlob as tb
from googletrans import Translator
import numpy as np
from unidecode import unidecode
%matplotlib inline

In [9]:
import flair

In [2]:
 translator = Translator()

In [3]:
translator.translate('안녕하세요.')

In [4]:
consumer_key = 'P1HzTo2vebuojpEzgfa6219Pn'
consumer_secret = 'EcGFaCAqohZI67cpOI8YVPp4BvkJhPGKOxAFo3N1gmt94cuGgZ'
access_token = '1441065260165775360-KYTdYuvS1fu0f6Igsg3iy8t7hquAYJ'
access_token_secret = 'WIna34xVpRZ6WZAdlU2KwVqJ1yeMbET6KeEd29slqPZcw'


In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [6]:
numPos = 0
numNeg = 0
total = 0

In [7]:
#Buscando tweets
#for tweet in tweepy.Cursor(api.search,q="petr4", since = '2019-02-22',lang="pt").items(70):
#    textPT = unidecode(tweet.text)
#    trans = translator.translate(textPT)
#    sentiment = tb(trans.text)
#    print("[PT-BR] "+textPT)
 #   print("[EN]    "+trans.text)
  #  print("[Sent.] "+str(sentiment.polarity)+"\n")
   # #Estatísticas
    #total += 1
    #if sentiment.polarity > 0: 
     #   numPos += 1 
    #elif sentiment.polarity < 0: 
     #   numNeg += 1

In [17]:
#Buscando tweets
probs = []
sentiments = []
for tweet in tweepy.Cursor(api.search,q="petr4", since = '2019-02-22',lang="pt").items(1000):
    textPT = unidecode(tweet.text)
    trans = translator.translate(textPT)
    sentiment = unidecode(trans.text)
    sentence = flair.data.Sentence(sentiment)
    print(sentence)
    sentiment_model.predict(sentence)
    # extract sentiment prediction
    probs.append(sentence.labels[0].score)  # numerical score 0-1
    sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'
    #Estatísticas
    total += 1
    if sentiment.polarity > 0: 
        numPos += 1 
    elif sentiment.polarity < 0: 
        numNeg += 1

In [49]:
#Sentimento geral
mediaPos = numPos/total
mediaNeg = numNeg/total
print('Porcentagem de comentários positivos: '+str(mediaPos))
print('Porcentagem de comentários negativos: '+str(mediaNeg))

Porcentagem de comentários positivos: 0.25984251968503935
Porcentagem de comentários negativos: 0.2559055118110236


In [46]:
analysis = None

In [47]:
tweets = [] # Lista vazia para armazenar scores
for tweet in public_tweets:
    print(tweet.text)
    analysis = tb(tweet.text)
    polarity = analysis.sentiment.polarity
    tweets.append(polarity)
    print(polarity)

NameError: name 'public_tweets' is not defined

In [50]:
print('MÉDIA DE SENTIMENTO: ' + str(np.mean(tweets)))

MÉDIA DE SENTIMENTO: 0.0290301452020202


In [45]:
df = pd.DataFrame()
for tweet in response.json()['statuses']:
    row = get_data(tweet)
    df = df.append(row, ignore_index=True)

In [23]:
df

,id,created_at,text
0,1441111837831954438,Thu Sep 23 18:46:51 +0000 2021,@JanainaDoBrasil Engraçado que quando a Lava J...
1,1441111495866142735,Thu Sep 23 18:45:29 +0000 2021,RT @_makavelijones: Presidente da Petrobras te...
2,1441111294648586240,Thu Sep 23 18:44:41 +0000 2021,"RT @alex_rangel_br: ""O Complexo Lagoon é alvo ..."
3,1441111294577283079,Thu Sep 23 18:44:41 +0000 2021,@g1 @mtst Enquanto o mundo está voando! Soluçã...
4,1441111241078894597,Thu Sep 23 18:44:29 +0000 2021,RT @cafecomferri: Narrativas retomadas sempre ...
...,...,...,...
95,1441092869746413576,Thu Sep 23 17:31:29 +0000 2021,"PARA DE ME MOSTRAR PINGUIM PETROBRAS, QUE PORRA"
96,1441092437955407885,Thu Sep 23 17:29:46 +0000 2021,RT @cafecomferri: Narrativas retomadas sempre ...
97,1441092345940742149,Thu Sep 23 17:29:24 +0000 2021,"RT @petrobras: Você sabia que, com o nosso apo..."
98,1441092318942040077,Thu Sep 23 17:29:17 +0000 2021,"RT @alex_rangel_br: ""O Complexo Lagoon é alvo ..."
